### 读取背景音乐

In [1]:
import wave
import numpy as np

# music = wave.open(r'background.wav', mode='rb')
# music.getparams()
# bg_data = music.readframes(nframes=32191488)
# bg_data = np.frombuffer(bg_data, dtype=np.int16)#音频的格式通常都是16位；buffer缓冲器
# bg_data.shape

# bg_data = np.zeros([44100*15*60], dtype=np.int16)

In [2]:
ding = wave.open(r'ding.wav', mode='rb')
ding.getparams()

_wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=39710, comptype='NONE', compname='not compressed')

In [3]:
nframes = 39710
nchannels=1
ding_data = ding.readframes(nframes=nframes)
ding_data = np.frombuffer(ding_data, dtype=np.int16)
# ding_data = ding_data.reshape(nframes, 2)
ding_data.shape

(39710,)

### 读取视频息肉出现时间点

In [4]:
import sys
import os
import subprocess
sys.path.append(r'../../NLP/MedicalRecord/Lib')

from MRRecordUtil import load_sheet_arr_dict

In [14]:
# 计算视频秒数
video_fps = 18
def calc_video_len(video_name, base_dir):
    img_folder = os.path.join(os.path.join(base_dir, 'images_wxr'), video_name)
    return len(os.listdir(img_folder)) // video_fps

# 合成视频音频
def gen_audio(ding_data, video_secs, framerate, polp_time_arr):
    print(video_secs, framerate, video_secs*framerate)
    print(polp_time_arr)
    bg_data = np.zeros([video_secs*framerate], dtype=np.int16)
    bg_len = bg_data.shape[0]
    start_frames = []
    if ding_data is not None:
        ding_len = ding_data.shape[0]
#         ding_sec = ding_len / framerate
#         print(ding_len)
        print('bg_len: %s, ding_len: %s' % (bg_len, ding_len))
        last_frame = -1
        for ptime in polp_time_arr:
            p_frame = int(ptime * framerate)
            if (last_frame == -1 or p_frame - last_frame > ding_len) and p_frame < bg_len:
                start_frame = p_frame
                end_frame = start_frame + ding_len
                end_frame = end_frame if bg_len > end_frame else bg_len
#                 print(ptime, start_frame, end_frame)
#                 bg_data[start_frame:end_frame, :] = ding_data[0:(end_frame-start_frame), :]
                bg_data[start_frame:end_frame] = ding_data[0:(end_frame-start_frame)]
                last_frame = p_frame
                start_frames.append(start_frame)
            else:
                print(ptime, p_frame - last_frame)
    print(start_frames)
    print(len(polp_time_arr), len(start_frames))
    print([f / framerate for f in start_frames])
    return bg_data

# 写音频
def write_audio(audio_data, secs, framerate=44100, nchannels=1, sampwidth=2):
    if os.path.exists('./generated.wav'):
        os.remove('./generated.wav')
    fp = wave.Wave_write('./generated.wav')
    #设置fp的参数
    fp.setframerate(framerate)
    fp.setnframes(secs*framerate)
    fp.setnchannels(nchannels)
    fp.setsampwidth(sampwidth)

    #写入目录
    fp.writeframes(audio_data.tobytes())
    fp.close()
    
# 读取excel记录视频数据
def get_video_data_from_sheet(sheet_data):
    video_data = []
    for r in sheet_data:
        r_data = {}
        r_data['视频名'] = '%d' % r['视频名']
        r_data['所选参数'] = r['所选参数']
#         r_data['假阳性出现时间'] = [float(e) for e in r['假阳性出现时间'].split('\n') if e != '']
#         r_data['真阳性出现时间'] = [float(e.split('-')[0]) / 18 for e in r['真阳性出现帧'].split('\n') if e != '']
#         r_data['息肉出现时间点'] = sorted(r_data['假阳性出现时间'] + r_data['真阳性出现时间'])
        r_data['息肉出现时间点'] = [float(e.split(' ')[0]) for e in r['假阳性出现时间_开始帧_结束帧'].split('\n') if e != '']
        video_data.append(r_data)
    return video_data

def add_audio_for_video_data(v_data, video_folder, base_dir):
    video_secs = calc_video_len(v_data['视频名'], base_dir)
    print('video seconds: %s' % video_secs)
    audo_data = gen_audio(ding_data, video_secs=video_secs, framerate=44100, polp_time_arr=v_data['息肉出现时间点'])
    write_audio(audo_data, video_secs, framerate=44100)
    
    video_path = os.path.join(video_folder, '%s_%s.avi' % (v_data['视频名'], v_data['所选参数']))
    trans_video_path = os.path.join(video_folder, '%s_%s_audio.mp4' % (v_data['视频名'], v_data['所选参数']))
    cmd = 'ffmpeg -i generated.wav -i %s %s' % (video_path, trans_video_path)
    subprocess.call(cmd)

In [15]:
excel = '息肉出现时间点统计（无真息肉）.xls'
for base_dir in [r'D:\项目资料\息肉假阳性\20221007']:
    video_folder = os.path.join(base_dir, 'videos_gen_wxr')
    sheet_data = load_sheet_arr_dict(os.path.join(base_dir, excel), 'Sheet1')
    video_data = get_video_data_from_sheet(sheet_data)
    
    for v_data in video_data:
        if len(v_data['视频名']) > 0:# and v_data['视频名'] in ['1307'] and v_data['所选参数'] == 0.02:
            print('processing ', v_data['视频名'], v_data['所选参数'])
            add_audio_for_video_data(v_data, video_folder, base_dir)


processing  3398 0.05
video seconds: 299
299 44100 13185900
[9.333, 13.944, 153.889, 155.167, 158.444, 160.5, 183.167, 183.944, 188.056, 189.722, 194.222, 203.444, 204.889, 210.444, 213.056, 223.778, 224.611, 225.556, 227.722, 228.833, 230.778, 232.056, 234.278, 235.111, 235.889, 240.444, 240.944, 241.833, 243.944, 246.222, 247.333, 252.0, 254.444, 258.889, 259.444, 267.667, 268.389, 270.222, 274.333, 277.889, 279.944, 283.722, 284.889, 290.889, 291.889]
bg_len: 13185900, ding_len: 39710
183.944 34266
224.611 36736
235.111 36736
240.944 22050
259.444 24476
268.389 31840
[411585, 614930, 6786504, 6842864, 6987380, 7078050, 8077664, 8293269, 8366740, 8565190, 8971880, 9035604, 9280580, 9395769, 9868609, 9947019, 10042540, 10091535, 10177309, 10233669, 10331659, 10402704, 10603580, 10664835, 10757930, 10858390, 10907385, 11113200, 11220980, 11417004, 11804114, 11916790, 12098085, 12254904, 12345530, 12512140, 12563604, 12828204, 12872304]
45 39
[9.332993197278912, 13.943990929705215, 153.

processing  3416 0.015
video seconds: 448
448 44100 19756800
[1.278, 27.944, 55.111, 56.056, 63.722, 65.056, 66.167, 74.111, 76.111, 90.111, 93.389, 96.444, 120.222, 128.667, 129.722, 130.833, 178.944, 181.389, 198.833, 201.333, 201.833, 215.667, 231.944, 232.944, 234.556, 235.278, 237.333, 247.833, 248.611, 254.889, 298.278, 315.056, 360.167, 380.556, 383.556, 404.167, 421.222, 423.833, 427.722, 443.0]
bg_len: 19756800, ding_len: 39710
201.833 22050
235.278 31840
248.611 34310
[56359, 1232330, 2430395, 2472069, 2810140, 2868969, 2917964, 3268295, 3356495, 3973895, 4118454, 4253180, 5301790, 5674214, 5720740, 5769735, 7891430, 7999254, 8768535, 8878785, 9510914, 10228730, 10272830, 10343919, 10466385, 10929435, 11240604, 13154059, 13893969, 15883364, 16782519, 16914819, 17823764, 18575890, 18691035, 18862540, 19536300]
40 37
[1.2779818594104309, 27.943990929705215, 55.110997732426306, 56.05598639455782, 63.72199546485261, 65.05598639455782, 66.16698412698413, 74.1109977324263, 76.11099

processing  3398 0.015
video seconds: 299
299 44100 13185900
[9.333, 10.611, 13.889, 16.167, 48.722, 61.056, 62.111, 72.889, 152.111, 153.833, 155.111, 160.389, 163.444, 163.889, 168.444, 171.167, 171.611, 172.444, 173.444, 179.778, 183.0, 183.944, 188.056, 189.722, 194.222, 202.556, 203.444, 204.889, 210.222, 211.167, 213.056, 214.167, 216.222, 217.944, 223.778, 224.611, 225.556, 230.778, 231.722, 235.889, 240.389, 247.333, 251.333, 252.0, 253.5, 258.167, 258.778, 259.444, 267.667, 268.222, 270.222, 272.5, 274.167, 276.944, 277.778, 283.722, 284.889, 285.944, 289.056, 289.944, 290.889, 294.056, 295.944]
bg_len: 13185900, ding_len: 39710
163.889 19624
171.611 19581
203.444 39161
224.611 36736
252.0 29415
258.778 26945
268.222 24476
277.778 36779
289.944 39161
[411585, 467945, 612504, 712964, 2148640, 2692569, 2739095, 3214404, 6708095, 6784035, 6840395, 7073154, 7207880, 7428380, 7548464, 7604780, 7648880, 7928209, 8070300, 8111930, 8293269, 8366740, 8565190, 8932719, 9035604, 9270790,

processing  3415 0.01
video seconds: 576
576 44100 25401600
[0.056, 6.056, 9.444, 14.889, 17.5, 21.778, 22.333, 27.667, 31.833, 35.333, 36.056, 39.833, 40.889, 43.167, 45.278, 51.611, 84.556, 87.444, 103.667, 110.167, 128.556, 132.833, 141.5, 142.611, 146.556, 147.333, 159.667, 166.167, 166.778, 185.667, 186.333, 188.444, 189.222, 204.111, 207.333, 212.222, 218.5, 223.889, 225.556, 242.611, 243.167, 244.889, 246.389, 289.778, 322.278, 323.056, 327.667, 328.278, 340.833, 350.722, 370.5, 381.222, 386.0, 388.556, 395.722, 400.389, 403.167, 408.889, 415.389, 417.833, 420.722, 427.889, 433.944, 441.222, 444.611, 450.778, 452.889, 453.778, 458.722, 461.056, 464.778, 466.333, 469.222, 473.556, 474.444, 486.667, 487.222, 488.111, 489.111, 491.778, 493.278, 494.722, 498.056, 501.833, 502.778, 505.0, 505.722, 507.722, 508.889, 515.111, 517.611, 519.444, 520.778, 528.389, 530.167, 532.167, 533.778, 535.611, 537.056, 538.833, 539.944, 540.722, 542.111, 547.333, 548.833, 553.389, 554.333, 564.889, 

processing  3429 0.015
video seconds: 347
347 44100 15302700
[0.056, 5.556, 9.444, 12.667, 19.222, 31.389, 33.778, 53.444, 55.5, 65.5, 70.444, 76.944, 120.278, 124.111, 124.833, 127.889, 129.389, 130.667, 135.389, 137.833, 142.389, 145.056, 148.778, 158.167, 161.556, 164.667, 170.778, 173.944, 182.167, 186.056, 187.056, 188.889, 192.167, 193.944, 207.667, 208.722, 209.333, 210.167, 219.778, 221.167, 222.667, 228.444, 229.111, 237.111, 242.444, 250.333, 253.833, 261.944, 262.944, 263.667, 264.167, 271.556, 277.611, 279.5, 282.333, 284.278, 286.389, 287.611, 289.667, 295.667, 298.611, 301.111, 302.778, 305.5, 308.111, 308.667, 310.167, 311.722, 314.833, 316.111, 317.056, 318.944, 319.556, 320.278, 329.5, 332.278, 336.556, 337.056, 341.556, 342.889]
bg_len: 15302700, ding_len: 39710
124.833 31840
209.333 26945
229.111 29415
263.667 31884
308.667 24519
319.556 26989
337.056 22050
[2469, 245019, 416480, 558614, 847690, 1384254, 1489609, 2356880, 2447550, 2888550, 3106580, 3393230, 5304259, 

In [11]:
704 * 44100

31046400